In [ ]:
# Adapted from Sionna's "Hello World" Example Notebook: https://nvlabs.github.io/sionna/examples/Hello_World.html
# Usage: python model.py [xml scene name] [starting frame]

print("Importing libraries")

import matplotlib.pyplot as plt
import numpy as np
import os
import sionna
from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera
import sys

os.chdir("/home/aidan/code/mv-obj-detection/training-data")

In [ ]:
SCATTERING = True
DIFFRACTION = True
SCATTERING_COEFFICIENTS = {"itu_concrete": 0.2, "itu_medium_dry_ground": 0.3, "itu_metal":0.02, "vacuum":0}

print("Setting CUDA variables")

if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load from parameters the scene name (for finding the xml file) and frame number to start at (defaults to 0)
SCENE_NAME = "scene3_solid_drone"
START_NUM = 0
print(f"Scene: {SCENE_NAME}, Starting frame: {START_NUM}")

scene = load_scene(f"blender/exported-scenes/{SCENE_NAME}/{SCENE_NAME}.xml")
scene.frequency = 2.4e9

# Print the radio materials in the scene for debugging
print("Materials in scene: ")
for i, obj in enumerate(scene.objects.values()):
    print(f"    {obj.name}: {obj.radio_material.name}")
    if SCATTERING:
        obj.radio_material.scattering_coefficient = SCATTERING_COEFFICIENTS[obj.radio_material.name]
        obj.radio_material.scattering_pattern = sionna.rt.DirectivePattern(alpha_r=4)

# Setup the transmitter
scene.tx_array = PlanarArray(
    num_rows=1,
    num_cols=1,
    vertical_spacing=0.5,
    horizontal_spacing=0.5,
    pattern="iso",
    polarization="V"
)

# Setup the receiver
scene.rx_array = PlanarArray(
    num_rows=2,
    num_cols=2,
    vertical_spacing=1, # Wavelengths
    horizontal_spacing=1, # Wavelengths
    pattern="iso",
    polarization="V"
)

# Position the authorized drones
scene.objects["auth_drone_0"].position = [ -5.0, -8.66, 20.0]
scene.objects["auth_drone_1"].position = [ -5.0,  8.66, 20.0]
scene.objects["auth_drone_2"].position = [ 10.0,  0.00, 20.0]

# Create and position the transmitters for each authorized drone
tx1 = Transmitter("tx1", scene.objects["auth_drone_0"].position + np.array([0,0,-0.25]), [0.0, 0.0, 0.0])
tx2 = Transmitter("tx2", scene.objects["auth_drone_1"].position + np.array([0,0,-0.25]), [0.0, 0.0, 0.0])
tx3 = Transmitter("tx3", scene.objects["auth_drone_2"].position + np.array([0,0,-0.25]), [0.0, 0.0, 0.0])

# Create and position the receiver
rx = Receiver   ("rx", [0.0, 0.0, 0.5], [0.0, 0.0, 0.0])

# Add the transmitters and receiver to the scene
scene.add(tx1)
scene.add(tx2)
scene.add(tx3)
scene.add(rx)

# Load in the numpy flightpath
flightpath = np.load("flightpaths/longlines_path1.npy")

def interpolate_path(flightpath):
  """ Takes a flightpath file and interpolates the position of the drone at each frame.
  
  Flightpath file is a list of [time, x position, y position, z position] samples at random
  positions in a square plane below the transmitting authorized drones and above the
  receiver on the ground.
  """

  START_TIMESTAMP = 0
  END_TIMESTAMP = 1000 # longlines_path1 goes from 0 to 1000 seconds long
  NUM_FRAMES = 20_000 # There should be 20,000 frames (20 frames per second)
  
  interp = lambda dimension_idx: np.interp(np.linspace(START_TIMESTAMP, END_TIMESTAMP, NUM_FRAMES), flightpath[:,0], flightpath[:,dimension_idx])
  interpolated_path = np.dstack([interp(1),interp(2),interp(3)])
  return interpolated_path[0]

# Copmute the path of the unauthroized drone. Array shape: (frames, x position, y position, z position)
interpolated_path = interpolate_path(flightpath)

SCENE_NAME = "solid_multidrone_10E"

# Move grid out of the way
# for obj in [f"location_grid_00{i}" for i in range(8)]:
#     scene.objects[obj].position = np.array([*scene.objects[obj].position[:2], -10000])

# scene.objects["building_0"].position = np.array([0,0,-1000])
# scene.objects["building_1"].position = np.array([0,0,-1000])
# scene.objects["building_2"].position = np.array([0,0,-1000])

# Iterate through each frame:
for frame in range(START_NUM, interpolated_path.shape[0]):
    print(f"Computing frame {frame} of {interpolated_path.shape[0]}")
    
    # Move unauthorized drone position
    scene.objects["unauth_drone"].position = np.array([interpolated_path[frame][0], interpolated_path[frame][1], 10.0])
    # Compute paths for the current scene
    paths = scene.compute_paths(
        scattering = True,
        diffraction = DIFFRACTION,
        max_depth = 3,
        check_scene = False,
        num_samples = 1e6,
        scat_keep_prob = 0.005,
    )
    break
print("Done.")

In [ ]:
# print("Loading Preview")
# labels = np.load("data/longlines_path1_labels.npz")["labels"]
# for obj in [f"location_grid_00{i}" for i in range(8)]:
#     scene.objects[obj].position = np.array([*scene.objects[obj].position[:2], 10])
print(paths.a.shape)
scene.preview(paths=paths)

In [ ]:
paths.types
# # plt.plot(np.squeeze(paths.objects).reshape(9,-1).swapaxes(0,1)/10, 'r')
# plt.plot(np.abs(np.squeeze(paths.a).reshape(12,-1).swapaxes(0,1))*1e4, 'g')
# plt.show()
# a, tau = paths.cir()
# a2, tau2 = paths_more_scatter.cir()
# a2, tau2 = np.squeeze(a2), np.squeeze(tau2)
# print(a2.shape)

# print(np.sum(paths_more_scatter.types == paths.LOS))
# print(np.mean(a2[:,:,paths_more_scatter.types.numpy().flatten() == paths.LOS], axis=-1))
# print(np.sum(paths_more_scatter.types == paths.SPECULAR))
# print(np.mean(a2[:,:,paths_more_scatter.types.numpy().flatten() == paths.SPECULAR], axis=-1))
# print(np.sum(paths_more_scatter.types == paths.DIFFRACTED))
# print(np.mean(a2[:,:,paths_more_scatter.types.numpy().flatten() == paths.DIFFRACTED], axis=-1))
# print(np.sum(paths_more_scatter.types == paths.SCATTERED))
# print(np.mean(a2[:,:,paths_more_scatter.types.numpy().flatten() == paths.SCATTERED], axis=-1))

# x = np.array(paths_more_scatter.types)
# x[0,128:138] = 5
# plt.imshow(x, aspect="auto", interpolation="none")
# plt.show()




# # # frame = 753
# # # scene.objects["unauth_drone"].position = np.array([4,-4,10])

# # # print(
# # # a, tau = paths.cir()
# # # print(a.shape)
# # # print(np.sum(np.squeeze(a)[0,0]==0))

# # # sionna.rt.camera.Camera
# # # scene.background = 
# # scene.render_to_file(filename="scene1.png", camera="preview")

In [ ]:
a2, tau2 = paths_more_scatter.cir()
help(sionna.rt.Paths.pad_or_crop)
sionna.rt.Paths.pad_or_crop(a=a2, tau=tau2, k=128)